In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
import time
import shutil
import os

# S is the set of selected configuractions that contribute to the cost function
# SS is the special configuration, that we would manually add to selected configurations. Since it turns out the training works when the selected configurations contains some special ones. 
S = np.load('data/S20_30.npy')
SS = np.array([[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1],
               [-1, +1, -1, -1, -1, -1, -1, +1, -1, -1, -1, -1, -1, +1, -1, -1, -1, -1],
               [-1, -1, -1, -1, -1, -1, -1, -1, -1, +1, +1, +1, -1, -1, -1, -1, -1, -1],
               [-1, +1, -1, -1, -1, -1, -1, +1, -1, +1, +1, +1, -1, +1, -1, -1, -1, -1]])

#This code is to search solution of RBM. The random initial parameters are stored in the variables. 
Variables = np.load('data/Variables.npy')


# pr and prr are the shortcut for cosh functions.
def pr(b, w, s1, s2, s3, s4):
    result = torch.cosh(b[0] + 1j * (w[0] * s1 + w[1] * s2 + w[2] * s3 + w[3] * s4))
    return result
def prr(b, b1, b2, w, s1, s2, s3):
    result = torch.cosh(1j * b + (b1 + b2 * 1j) + 1j * w * (s1 + s2 + s3))
    return result
def prrr(b, b1, b2, w, s1, s2, s3, s4, s5, s6):
    result = torch.cosh(1j * b + (b1 + b2 * 1j) + 1j * w * (s1 + s2 + s3 + s4 + s5 + s6))
    return result


# ph is the phase function given by the form of RBM.
def ph(bl, phase, wl, al, bv, wv, bf, wf, s):
    result = (np.exp(1j * GL[0]) * np.exp(1j * GL[1]) * np.exp(1j * GL[2])
              * torch.exp(1j * al[0] * (s[0]+s[1]+s[2])) * torch.exp(1j * al[1] * (s[3]+s[9]+s[15]))
              * torch.exp(1j * al[2] * (s[0]+s[1]+s[2]+s[3]+s[9]+s[15]))
              * prr(bl[0], phase[0], phase[3], wl[0], s[0], s[1], s[2])
              * prr(bl[1], phase[1], phase[4], wl[1], s[3], s[9], s[15])
              * prrr(bl[2], phase[2], phase[5], wl[2], s[0], s[1], s[2], s[3], s[9], s[15])
              * pr(bv, wv, s[0], s[3], s[2], s[15]) * pr(bv, wv, s[1], s[4], s[0], s[16])
              * pr(bv, wv, s[2], s[5], s[1], s[17]) * pr(bv, wv, s[6], s[9], s[8], s[3])
              * pr(bv, wv, s[7], s[10], s[6], s[4]) * pr(bv, wv, s[8], s[11], s[7], s[5])
              * pr(bv, wv, s[12], s[15], s[14], s[9]) * pr(bv, wv, s[13], s[16], s[12], s[10])
              * pr(bv, wv, s[14], s[17], s[13], s[11]) * pr(bf, wf, s[4], s[6], s[3], s[0])
              * pr(bf, wf, s[5], s[7], s[4], s[1]) * pr(bf, wf, s[3], s[8], s[5], s[2])
              * pr(bf, wf, s[10], s[12], s[9], s[6]) * pr(bf, wf, s[11], s[13], s[10], s[7])
              * pr(bf, wf, s[9], s[14], s[11], s[8]) * pr(bf, wf, s[16], s[0], s[15], s[12])
              * pr(bf, wf, s[17], s[1], s[16], s[13]) * pr(bf, wf, s[15], s[2], s[17], s[14]))
    return result


# nor is the L_1 normalization function. 
def nor(bl, phase, wl, al, bv, wv, bf, wf):
    result = 0
    for i in range(len(S)):
      result += abs(ph(bl, phase, wl, al, bv, wv, bf, wf, S[i]))
    return result


# Define the action of 9 vertex operators.
A1 = np.diag([-1, +1, -1, -1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, -1, +1, +1])
A2 = np.diag([-1, -1, +1, +1, -1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, -1, +1])
A3 = np.diag([+1, -1, -1, +1, +1, -1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, -1])
A4 = np.diag([+1, +1, +1, -1, +1, +1, -1, +1, -1, -1, +1, +1, +1, +1, +1, +1, +1, +1])
A5 = np.diag([+1, +1, +1, +1, -1, +1, -1, -1, +1, +1, -1, +1, +1, +1, +1, +1, +1, +1])
A6 = np.diag([+1, +1, +1, +1, +1, -1, +1, -1, -1, +1, +1, -1, +1, +1, +1, +1, +1, +1])
A7 = np.diag([+1, +1, +1, +1, +1, +1, +1, +1, +1, -1, +1, +1, -1, +1, -1, -1, +1, +1])
A8 = np.diag([+1, +1, +1, +1, +1, +1, +1, +1, +1, +1, -1, +1, -1, -1, +1, +1, -1, +1])
A9 = np.diag([+1, +1, +1, +1, +1, +1, +1, +1, +1, +1, +1, -1, +1, -1, -1, +1, +1, -1])
# The Si below represents the configuration that is acted by the i-th A operator
S1 = np.dot(S, A1)
S2 = np.dot(S, A2)
S3 = np.dot(S, A3)
S4 = np.dot(S, A4)
S5 = np.dot(S, A5)
S6 = np.dot(S, A6)
S7 = np.dot(S, A7)
S8 = np.dot(S, A8)
S9 = np.dot(S, A9)

# lam is the multiplier to amplify the choice of ground state
# Define the cost function
def criterion(bl, phase, wl, al, bv, wv, bf, wf):
    V = torch.empty((18*N+3,), dtype=torch.complex64)
    for k in range(N):
        sub = ph(bl, phase, wl, al, bv, wv, bf, wf, S[k])
        one = ph(bl, phase, wl, al, bv, wv, bf, wf, SS[0])
        V[k] = ph(bl, phase, wl, al, bv, wv, bf, wf, S1[k]) - sub
        V[1*N+k] = ph(bl, phase, wl, al, bv, wv, bf, wf, S2[k]) - sub
        V[2*N+k] = ph(bl, phase, wl, al, bv, wv, bf, wf, S3[k]) - sub
        V[3*N+k] = ph(bl, phase, wl, al, bv, wv, bf, wf, S4[k]) - sub
        V[4*N+k] = ph(bl, phase, wl, al, bv, wv, bf, wf, S5[k]) - sub
        V[5*N+k] = ph(bl, phase, wl, al, bv, wv, bf, wf, S6[k]) - sub
        V[6*N+k] = ph(bl, phase, wl, al, bv, wv, bf, wf, S7[k]) - sub
        V[7*N+k] = ph(bl, phase, wl, al, bv, wv, bf, wf, S8[k]) - sub
        V[8*N+k] = ph(bl, phase, wl, al, bv, wv, bf, wf, S9[k]) - sub
        V[9*N+k] = (S[k][4]*S[k][6]*S[k][3]*S[k][0]-1) * sub
        V[10*N+k] = (S[k][5]*S[k][7]*S[k][4]*S[k][1]-1) * sub
        V[11*N+k] = (S[k][3]*S[k][8]*S[k][5]*S[k][2]-1) * sub
        V[12*N+k] = (S[k][10]*S[k][12]*S[k][9]*S[k][6]-1) * sub
        V[13*N+k] = (S[k][11]*S[k][13]*S[k][10]*S[k][7]-1) * sub
        V[14*N+k] = (S[k][9]*S[k][14]*S[k][11]*S[k][8]-1) * sub
        V[15*N+k] = (S[k][16]*S[k][0]*S[k][15]*S[k][12]-1) * sub
        V[16*N+k] = (S[k][17]*S[k][1]*S[k][16]*S[k][13]-1) * sub
        V[17*N+k] = (S[k][15]*S[k][2]*S[k][17]*S[k][14]-1) * sub
        V[18*N+0] = lam * (ph(bl, phase, wl, al, bv, wv, bf, wf, SS[1])-2*one)
        V[18*N+1] = lam * (ph(bl, phase, wl, al, bv, wv, bf, wf, SS[2])-3*one)
        V[18*N+2] = lam * (ph(bl, phase, wl, al, bv, wv, bf, wf, SS[3])-4*one)
    v = torch.norm(V)/nor(bl, phase, wl, al, bv, wv, bf, wf)
    return v

In [4]:
# Define important parameters. ss: step length, tt: total training time
# Select is to choose those with potential to be solution
# The start parameters are generated, so we can test at same start points.
start_time = time.time()
(ss, tt, lam, N) = (0.03, 10, 1.0, len(S))
BF = torch.tensor([0.0])
WF = torch.tensor([np.pi/4, np.pi/4, np.pi/4, np.pi/4])
GL = torch.tensor([np.pi/4, np.pi/4, np.pi/2])
BL = torch.tensor([np.pi/4, np.pi/4, 0.0])
WL = torch.tensor([np.pi/4, np.pi/4, np.pi/4])
AL = torch.tensor([np.pi/4, np.pi/4, np.pi/4])
All = []
Select = []
for k in range(25, 30):
    IN = Variables[k]
    BV = torch.tensor([IN[0]], requires_grad=True)
    WV = torch.tensor([IN[1], IN[2], IN[3], IN[4]], requires_grad=True)
    Phase = torch.tensor([IN[5], IN[6], IN[7], IN[8], IN[9], IN[10]], requires_grad=True)
    # We use Adam to do the gradient descent
    optimizer = torch.optim.Adam([BV, WV, Phase], lr=ss) 
    for h in range(tt):
        cost = criterion(BL, Phase, WL, AL, BV, WV, BF, WF)
        cost.backward()
        optimizer.step()
        optimizer.zero_grad()
        All.append(IN.tolist())
    print("%d: (%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f)->(%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f,%.2f), cost=%.2e" % (k, IN[0].tolist(), IN[1].tolist(), IN[2].tolist(), IN[3].tolist(),IN[4].tolist(), IN[5].tolist(), IN[6].tolist(), IN[7].tolist(),IN[8].tolist(), IN[9].tolist(), IN[10].tolist(),BV[0].tolist(), WV[0].tolist(), WV[1].tolist(), WV[2].tolist(), WV[3].tolist(),Phase[0].tolist(),Phase[1].tolist(),Phase[2].tolist(),Phase[3].tolist(),Phase[4].tolist(),Phase[5].tolist(),cost))
    if cost < 0.1:
        Select.append(IN.tolist())
end_time = time.time()
execution_time = end_time - start_time
print(f"Execution time: {execution_time} seconds")
print(f"Select:{Select}")


/tmp/ipykernel_14164/23672477.py:32: DeprecationWarning: __array_wrap__ must accept context and return_scalar arguments (positionally) in the future. (Deprecated NumPy 2.0)
  result = (np.exp(1j * GL[0]) * np.exp(1j * GL[1]) * np.exp(1j * GL[2])


25: (0.17,-1.09,1.06,-0.14,1.24,0.44,1.45,0.89,-0.33,-0.52,0.73)->(0.39,-1.27,0.88,-0.00,1.10,0.46,1.58,0.60,-0.34,-0.65,0.49), cost=1.09e+00
26: (-0.50,3.14,0.19,-1.36,2.27,-1.29,-0.95,-1.09,-0.44,0.87,0.26)->(-0.20,3.10,0.11,-1.57,1.97,-1.57,-0.88,-1.26,-0.55,0.59,0.50), cost=2.75e-01
27: (1.93,1.72,1.90,-3.08,-0.57,-1.18,0.78,0.71,-0.08,-0.18,-1.36)->(2.22,1.60,2.05,-3.20,-0.41,-0.88,0.48,1.00,0.05,0.08,-1.07), cost=2.87e-01
28: (-0.39,1.38,2.97,-2.08,-1.76,1.45,0.15,0.87,-1.48,-1.46,-1.53)->(-0.64,1.25,3.25,-2.36,-1.48,1.71,0.42,1.15,-1.18,-1.18,-1.23), cost=1.14e+00
29: (-0.17,1.98,-2.61,-2.46,0.48,0.76,0.82,-1.54,0.96,0.21,0.26)->(-0.46,2.00,-2.46,-2.22,0.63,0.98,0.93,-1.41,0.69,0.16,0.15), cost=9.75e-01
Execution time: 167.14213824272156 seconds
Select:[]


In [ ]:
26: (-0.50,3.14,0.19,-1.36,2.27,-1.29,-0.95,-1.09,-0.44,0.87,0.26)->(-0.01,3.12,0.00,-1.58,1.51,-0.66,-0.45,-1.17,0.00,0.00,0.01), cost=7.20e-03